### 1. 병합

##### 작업 방향

1. 모든 테이블은 '기준년월', '발급회원번호'를 중복?식별자로 해서 merge가 가능합니다.
2. '회원정보' 테이블은 7월에 vip였던 사람들의 추이를 볼 수 있도록 구성되어있습니다.
3. 따라서 '회원정보' 테이블을 기준으로 join(merge)하면, 7월에 vip였던 사람들의 정보를 모두 통합할 수 있을 거라 생각됩니다.
4. 컬럼 순서도 중요하긴 하니까 저의 통합하는 순서는 01(회원정보)에 02(신용정보)를 붙이고, 그 다음 03.. 마지막엔 08 붙도록 하면 될 것 같습니다.

[ 테이블 넘버링 ]

01 - 회원정보

02 - 신용정보

03 - 승인매출정보

04 - 청구정보

05 - 잔액정보

06 - 채널정보

07 - 마케팅정보

08 - 성과 정보

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
import warnings
warnings.filterwarnings('ignore')

##### raw data 처리:

In [16]:
# 최초 버전 (메모리 이슈 발생 가능성이 있음: 생략)

# df01 = pd.read_csv('./통합데이터셋/2018_회원정보_통합.csv',encoding='utf-8')
# df02 = pd.read_csv('./통합데이터셋/2018_신용정보_통합.csv',encoding='utf-8')
# df03 = pd.read_csv('./통합데이터셋/2018_승인매출정보_통합.csv',encoding='utf-8')
# df04 = pd.read_csv('./통합데이터셋/2018_청구정보_통합.csv',encoding='utf-8')
# df05 = pd.read_csv('./통합데이터셋/2018_잔액정보_통합.csv',encoding='utf-8')
# df06 = pd.read_csv('./통합데이터셋/2018_채널정보_통합.csv',encoding='utf-8')
# df07 = pd.read_csv('./통합데이터셋/2018_마케팅정보_통합.csv',encoding='utf-8')
# df08 = pd.read_csv('./통합데이터셋/2018_성과정보_통합.csv',encoding='utf-8')

# merged_df = df01

# df_list = [df02, df03, df04, df05, df06, df07, df08]

# for df_el in df_list:
#     merged_df = pd.merge(merged_df, df_el, on=['기준년월', '발급회원번호'], how='left')

# merged_df.to_csv('./신용카드VIP회원정보.csv', encoding='utf-8')

# 메모리 과부하를 줄이는 방식

# import pandas as pd
# import gc

# df_final = pd.read_csv('./통합데이터셋/2018_회원정보_통합.csv', encoding='utf-8')

# paths = [
#     './통합데이터셋/2018_신용정보_통합.csv',
#     './통합데이터셋/2018_승인매출정보_통합.csv',
#     './통합데이터셋/2018_청구정보_통합.csv',
#     './통합데이터셋/2018_잔액정보_통합.csv',
#     './통합데이터셋/2018_채널정보_통합.csv',
#     './통합데이터셋/2018_마케팅정보_통합.csv',
#     './통합데이터셋/2018_성과정보_통합.csv'
# ]
# for path in paths:
#     df_el = pd.read_csv(path, encoding='utf-8')
    
#     df_final = pd.merge(df_final, df_el, on=['기준년월', '발급회원번호'], how='left')
    
#     del df_el

#     gc.collect()

# df_final.to_csv('./신용카드VIP회원정보2.csv', encoding='utf-8')

In [4]:
# 메모리 과부하를 줄이는 방식

# import pandas as pd
# import gc

# df_final = pd.read_csv('./통합데이터셋/2018_회원정보_통합.csv', encoding='utf-8')

# paths = [
#     './통합데이터셋/2018_신용정보_통합.csv',
#     './통합데이터셋/2018_승인매출정보_통합.csv',
#     './통합데이터셋/2018_청구정보_통합.csv',
#     './통합데이터셋/2018_잔액정보_통합.csv',
#     './통합데이터셋/2018_채널정보_통합.csv',
#     './통합데이터셋/2018_마케팅정보_통합.csv',
#     './통합데이터셋/2018_성과정보_통합.csv'
# ]
# for path in paths:
#     df_el = pd.read_csv(path, encoding='utf-8')
    
#     df_final = pd.merge(df_final, df_el, on=['기준년월', '발급회원번호'], how='left')
    
#     del df_el

#     gc.collect()

# df_final.to_csv('./신용카드VIP회원정보2.csv', encoding='utf-8')

---

### 2. 만든 통합 파일(신용카드VIP회원정보.csv)의 전처리와 저장

In [2]:
df = pd.read_csv('../data/merged/신용카드VIP회원정보.csv', encoding = 'utf-8')

In [3]:
# 1. 제거할 컬럼 확인

# df.columns
    # '잔액_B0M_y', '연체일수_B2M_y', '월중평잔_y', '평잔_3M_y', '평잔_6M_y' 제거 후,
    # '잔액_B0M_x', '연체일수_B2M_x', '월중평잔_x', '평잔_3M_x', '평잔_6M_x' 의 각 컬럼명에서 _x를 제거
    # 'Unnamed: 0' 컬럼 제거

df.drop(columns=['잔액_B0M_y', '연체일수_B2M_y', '월중평잔_y', '평잔_3M_y', '평잔_6M_y'], inplace=True)

df.drop(columns=['Unnamed: 0'], inplace=True)

df.columns = df.columns.str.replace('_x', '')

df.columns

# 2. 나이 연령대 글자 제거하고 int 처리

df['연령'] = df['연령'].str.replace(r'\D', '', regex=True)

df['연령'] = df['연령'].astype(int)

# 3-1. Life_Stage의 이상치 제거 (= 연령대와 Life_Stage 정의가 매칭이 안 되는 경우)

# 20대와 30대 조건 : Life_Stage 값이 '5.자녀성장기(2)', '6.자녀출가기', '7.노령'에 해당하면 해당 열 제거
cond_2030 = df['연령'].isin([20, 30]) & df['Life_Stage'].isin(['5.자녀성장기(2)', '6.자녀출가기', '7.노령'])

# 40대 조건 : 40대는 Life_Stage 값이 '7.노령'에 해당하면 해당 열 제거
cond_40 = (df['연령'] == 40) & (df['Life_Stage'] == '7.노령')

# 50대, 60대, 70대 조건 : '1.Single', '2.가족형성기', '3.자녀출산기', '4.자녀성장기(1)'에 해당하면 해당 열 제거
cond_506070 = df['연령'].isin([50, 60, 70]) & df['Life_Stage'].isin(['1.Single', '2.가족형성기', '3.자녀출산기', '4.자녀성장기(1)'])

# 2. 모든 삭제 조건을 하나로 결합 (OR 조건)
drop_condition = cond_2030 | cond_40 | cond_506070

# 3. 조건에 해당하지 않는(~) 정상 데이터만 추출하여 덮어쓰기 (이상치 행 제거)
df = df[~drop_condition].copy()

# 3-1. (실행 안 해도 전처리에 이상 없음) 제대로 삭제되었는지 확인

base = df[["발급회원번호", "연령", "Life_Stage"]].drop_duplicates()
base

cnt = base.groupby(["연령", "Life_Stage"]).size().reset_index(name="건수")
cnt

total = cnt.groupby(["연령"])["건수"].sum().reset_index(name="전체건수")
total

result = cnt.merge(total, on="연령", how="left")
result

# 3-1. (계속) 6개월 동안의 데이터 중 하나의 컬럼이라도 빈 회원은 데이터 삭제

    # 1. 발급회원번호별 데이터 개수 집계
user_counts = df['발급회원번호'].value_counts()
print(user_counts)

    # 2. 데이터가 정확히 6개인 발급회원번호만 추출
valid_users = user_counts[user_counts == 6].index

    # 3. 해당 회원번호에 속하는 데이터만 남기기
df = df[df['발급회원번호'].isin(valid_users)].copy()

# 3-2. Life_Stage 컬럼을 활용한 파생변수- '양육자녀_유무' 생성

transform = {
    '1.Single': 0, 
    '2.가족형성기': 0, 
    '3.자녀출산기': 1, 
    '4.자녀성장기(1)': 1, 
    '5.자녀성장기(2)': 1, 
    '6.자녀출가기': 0, 
    '7.노령': 0
}

# map 함수를 적용하여 새로운 컬럼 생성
df['양육자녀_유무'] = df['Life_Stage'].map(transform)

df[['기준년월','발급회원번호', 'Life_Stage','양육자녀_유무']].head(10)

# 4. 이상치, 자료형 변경

# 4-1. '연체일수_B2M_x' -99999999

print(len(df[df['연체일수_B2M'] > 0])) # 21,426
print(len(df[df['연체일수_B2M'] < 0])) # 3,064,586
print(len(df[df['연체일수_B2M'] == 0])) # 499,540

df.loc[df['연체일수_B2M'] < 0, '연체일수_B2M'] = -1

print(len(df[df['연체일수_B2M'] < -1])) #0

# # 4-2. '기준년월'

# df['기준년월'] = df['기준년월'].astype(str) 
# print(type(df['기준년월'].iloc[0]))
# print(df['기준년월'])

# df['기준년월'] = pd.to_datetime(df['기준년월'].astype(str).str[:6], format='%Y%m')

# print(df['기준년월'].dtype) 

# # 옵션: 추가 날짜 (월, 년, 월년) 컬럼 만들기
# # df['year'] = df['기준년월'].dt.year
# # df['month'] = df['기준년월'].dt.month
# # df['year_month'] = df['기준년월'].dt.to_period('M')

# print(df['기준년월'])

발급회원번호
SYN_0          6
SYN_1999369    6
SYN_1999459    6
SYN_1999463    6
SYN_1999464    6
              ..
SYN_2369781    1
SYN_2570546    1
SYN_2685302    1
SYN_1331585    1
SYN_2421235    1
Name: count, Length: 597408, dtype: int64
21208
3060527
498183
0


In [ ]:
# 1인 값은 이탈자
df["y"] = (df["이용금액_신용_R12M"] == 0).astype(int)
df

# df.drop("Unnamed: 0", axis = 1)

df.to_csv('../data/merged/신용카드VIP회원정보.csv', index = False, encoding="UTF-8")

,기준년월,발급회원번호,연령,VIP등급코드,회원여부_이용가능,회원여부_연체,탈회횟수_누적,최종탈회후경과월,탈회횟수_발급1년이내,거주시도명,...,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_인터넷_B0M,컨택건수_이용유도_당사앱_B0M,증감율_이용건수_신용_전월,증감율_이용금액_신용_전월,증감율_이용건수_신용_분기,증감율_이용금액_신용_분기,변동률_잔액_B1M,양육자녀_유무,y
0,201807,SYN_0,40,7,1,0,1,61,0,경기,...,0,0,0,0.454549,0.409767,-0.476454,-0.411320,0.261886,1,0
1,201807,SYN_8,20,7,1,0,0,0,0,충남,...,0,1,0,-0.272107,0.137583,0.464710,0.228002,-0.241684,0,0
2,201807,SYN_9,30,6,1,0,0,0,0,경기,...,1,1,0,0.291142,0.329639,-0.345280,-0.061798,0.273881,1,0
3,201807,SYN_17,70,7,1,0,0,0,0,경기,...,0,1,0,-0.021992,0.304045,0.057209,-0.196456,0.131636,0,0
4,201807,SYN_18,40,7,1,0,0,0,0,경기,...,0,0,0,0.000000,0.000103,-2.000000,-2.000000,0.093721,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3585547,201812,SYN_2999978,40,6,1,0,0,0,0,경기,...,0,0,0,0.025192,0.239937,-0.126710,-0.222808,-0.051395,1,0
3585548,201812,SYN_2999982,40,7,1,0,1,51,0,서울,...,0,0,0,0.119454,0.267341,0.080545,0.002163,-0.013494,1,0
3585549,201812,SYN_2999984,30,7,1,0,0,0,0,경기,...,0,0,0,-0.003368,0.277482,-0.239642,-0.010877,0.013066,1,0
3585550,201812,SYN_2999990,30,7,1,0,1,48,0,서울,...,0,0,0,-0.108888,0.273545,0.193081,0.092169,-0.097501,0,0


In [5]:
X = df.drop(columns=["이용금액_신용_R12M", "y"])  # drop both
y = df["y"]

# Model 
- data split: group shuffle split

In [6]:
# korean font setting
import platform

if platform.system() == 'Darwin': # macOS
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')

plt.rcParams['axes.unicode_minus'] = False

In [7]:
len(df.columns)
len(X)

3579918

In [ ]:
print(f"데이터에있는 월들: {sorted(df['기준년월'].unique())}")
print("\n" + "=" * 60)
print("1. 이탈 ('y' 컬럼)")
print("=" * 60)
print(df['y'].value_counts())
print(f"전반적 이탈률 : {df['y'].mean():.3%}")
print(f"사용자당 평균 행 수: {len(df) / df['발급회원번호'].nunique()}")

Months in data: [np.int64(201807), np.int64(201808), np.int64(201809), np.int64(201810), np.int64(201811), np.int64(201812)]

1. CHURN LABEL (pre-built column 'y')
y
0    3550807
1      29111
Name: count, dtype: int64
Overall churn rate : 0.813%
Average Rows per user: 6.0


#### GroupShuffleSplit
- 1) 사용자 기준 분리 (User-based Split)
- 2) 사용자를 8:2 비율로 나눔
- 3) 처음 보는 유저에 대한 예측 평가
- 4) train, test에 유저 정보 믹스하면 안됨 

In [18]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GroupShuffleSplit, GroupKFold

user_groups = df['발급회원번호']

# OrdinalEncoder
cat_cols_to_encode = ['Life_Stage', '이용금액대']
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[cat_cols_to_encode] = enc.fit_transform(X[cat_cols_to_encode].astype(str))

X = X.drop(columns=['발급회원번호'], errors='ignore')

# 데이터 누수 처리
leakage_cols = [
    '이용후경과월_신용',
    '회원여부_이용가능',
    '회원여부_연체',
    '최종이용일자_신판',
    '기준년월',
    '최종탈회후경과월',
    '탈회횟수_누적',
    '탈회횟수_발급1년이내',
    '거주시도명',
    '신용카드_결제금액_B0M',
    '단기카드대출_이용금액_B0M',
]

print(f"X columns drop 전 : {X.shape[1]}")
X = X.drop(columns=leakage_cols, errors='ignore')
print(f"X columns drop 후: {X.shape[1]}")
print(f"남아있는 features    :\n{X.columns.tolist()}")

# GroupShuffleSplit 80/20
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=user_groups))

X_train = X.iloc[train_idx]
X_test  = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test  = y.iloc[test_idx]

print(f"  Train : {len(X_train):,} rows | Users: {user_groups.iloc[train_idx].nunique():,}")
print(f"  Test  : {len(X_test):,} rows  | Users: {user_groups.iloc[test_idx].nunique():,}")
print(f"  Train 이탈 rate : {y_train.mean():.2%}")
print(f"  Test  이탈 rate : {y_test.mean():.2%}")

# -4) train and test 에서 유저 겹침 없는지 확인
overlap = set(user_groups.iloc[train_idx]) & set(user_groups.iloc[test_idx])
assert len(overlap) == 0, f"User leakage detected: {len(overlap)} users in both sets"
print(f"  User overlap     : {len(overlap)}  ✓ (no leakage)")

# GroupKFold (5 fold)
gkf = GroupKFold(n_splits=5)
groups_train = user_groups.iloc[train_idx]

for fold, (tr, val) in enumerate(gkf.split(X_train, y_train, groups=groups_train)):
    val_churn = y_train.iloc[val].mean()    
    # 불균형의 출력해보기
    flag = " ⚠ imbalanced" if val_churn < 0.05 or val_churn > 0.5 else ""
    print(f"  Fold {fold+1}: train={len(tr):5,} rows | "
          f"val={len(val):5,} rows | "
          f"val churn(이탈={val_churn:.2%}{flag}")

X columns drop 전 : 52
X columns drop 후: 52
남아있는 features    :
['연령', 'VIP등급코드', '유효카드수_신용', '이용카드수_신용', '이용금액_R3M_신용', '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수', 'Life_Stage', 'CA이자율_할인전', '강제한도감액금액_R12M', '한도증액금액_R12M', '연체감액여부_R3M', '시장단기연체여부_R3M', '이용건수_신용_B0M', '이용금액_신용_B0M', '이용건수_신용_R12M', '이용가맹점수', '이용금액_업종기준', '이용금액_쇼핑', '이용금액_요식', '이용금액_교통', '이용금액_의료', '이용금액_납부', '이용금액_교육', '이용금액_여유생활', '이용금액_사교활동', '이용금액_일상생활', '이용금액_해외', 'RP건수_B0M', 'RP금액_B0M', '이용금액대', '포인트_잔여포인트_B0M', '마일_잔여포인트_B0M', '잔액_B0M', '연체일수_B2M', '월중평잔', '평잔_3M', '평잔_6M', '컨택건수_이용유도_TM_B0M', '컨택건수_이용유도_LMS_B0M', '컨택건수_이용유도_EM_B0M', '컨택건수_이용유도_청구서_B0M', '컨택건수_이용유도_인터넷_B0M', '컨택건수_이용유도_당사앱_B0M', '증감율_이용건수_신용_전월', '증감율_이용금액_신용_전월', '증감율_이용건수_신용_분기', '증감율_이용금액_신용_분기', '변동률_잔액_B1M', '양육자녀_유무']
  Train : 2,863,932 rows | Users: 477,322
  Test  : 715,986 rows  | Users: 119,331
  Train 이탈 rate : 0.82%
  Test  이탈 rate : 0.79%
  User overlap     : 0  ✓ (no leakage)
  Fold 1: train=2,291,142 rows | val=572,790 r

#### LightGBM 
- LightGBM: Gradient Boosted Decision Tree (GBDT) algorithm

In [19]:
from sklearn.metrics import confusion_matrix, classification_report

from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(
    n_estimators=1000,         
    learning_rate=0.01,
    num_leaves=31,
    max_depth=4,               # pre prenung
    min_child_samples=100,     # pre prenung
    min_split_gain=0.1,        # pre prenung
    class_weight='balanced',
    reg_alpha=1.0,             # L1
    reg_lambda=1.0,            # L2
    random_state=42
)
eval_set = [(X_train, y_train), (X_test, y_test)]
lgbm.fit(X_train, y_train, eval_set=eval_set)
lgbm.score(X_train, y_train), lgbm.score(X_test, y_test)
y_pred = lgbm.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Confusion Matrix:
 [[709825    469]
 [     5   5687]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    710294
           1       0.92      1.00      0.96      5692

    accuracy                           1.00    715986
   macro avg       0.96      1.00      0.98    715986
weighted avg       1.00      1.00      1.00    715986



#### LightGBM Optuna 활용
- study에서  (optuna.create_study 함수 사용) 최고  F1-Score,  parameters  출력

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import optuna
from sklearn.model_selection import GroupKFold, cross_val_score
from lightgbm import LGBMClassifier

def objective(trial):
    param = {
        "n_estimators": 200, # 전체 트리의 개수 설정
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.05, log=True), 
        "num_leaves": trial.suggest_int("num_leaves", 15, 63), # 최대 잎(leaf) 
        "max_depth": trial.suggest_int("max_depth", 3, 8), # 깊이 
        "min_child_samples": trial.suggest_int("min_child_samples", 50, 500), # 리프 노드가 되기 위해 필요한 최소 데이터 개수 
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0), # 분할을 수행하기 위해 필요한 최소 손실 감소 값
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0), # L1 가중치 규제
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 50.0), # L2 가중치 규제 
        "subsample": trial.suggest_float("subsample", 0.6, 1.0), # 샘플링 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 0.9), # 피처(변수) 샘플링 비율
        "class_weight": "balanced", # 불균형 해결
        "random_state": 42, # 난수 고정
    }

    model = LGBMClassifier(**param)
    
    # GroupKFold
    gkf = GroupKFold(n_splits=3)
    
    # 최적화 F1-score 
    score = cross_val_score(
        model, X_train, y_train, 
        groups=user_groups.iloc[train_idx], 
        cv=gkf, 
        scoring='f1'
    ).mean()
    
    return score



# study 생성
study = optuna.create_study(direction="maximize")

# 30번 study 실행
study.optimize(objective, n_trials=30)

print("Best F1-Score:", study.best_value)
print("Best Parameters:", study.best_params)


# 최적화 parameters로 모델 생성 
best_lgbm = LGBMClassifier(**study.best_params, class_weight='balanced', random_state=42)

# 학습
best_lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test)])

# 평가
y_pred = best_lgbm.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[I 2026-02-24 09:06:03,677] A new study created in memory with name: no-name-4bd5f887-8b2d-43cf-9c20-a20277397bab


[I 2026-02-24 09:06:19,548] Trial 0 finished with value: 0.956435844115972 and parameters: {'learning_rate': 0.023770116390649153, 'num_leaves': 45, 'max_depth': 3, 'min_child_samples': 76, 'min_split_gain': 0.07603286851468805, 'reg_alpha': 4.33332697803067, 'reg_lambda': 2.700849799148878, 'subsample': 0.6344900221305567, 'colsample_bytree': 0.8036884372937585}. Best is trial 0 with value: 0.956435844115972.
[I 2026-02-24 09:06:37,848] Trial 1 finished with value: 0.9435014289524629 and parameters: {'learning_rate': 0.006432976727468349, 'num_leaves': 46, 'max_depth': 4, 'min_child_samples': 62, 'min_split_gain': 0.9558196870906221, 'reg_alpha': 6.590509211843326, 'reg_lambda': 28.363284254880366, 'subsample': 0.6061051446315852, 'colsample_bytree': 0.6278343066901366}. Best is trial 0 with value: 0.956435844115972.
[I 2026-02-24 09:06:52,564] Trial 2 finished with value: 0.8451032356162594 and parameters: {'learning_rate': 0.0020360087068792437, 'num_leaves': 30, 'max_depth': 5, 'mi

Best F1-Score: 0.9627424002019128
Best Parameters: {'learning_rate': 0.03454910980155507, 'num_leaves': 39, 'max_depth': 6, 'min_child_samples': 137, 'min_split_gain': 0.21730280963923776, 'reg_alpha': 0.1809617303491019, 'reg_lambda': 0.6799496894223173, 'subsample': 0.7922766484407321, 'colsample_bytree': 0.8466106278097849}
[[709721    573]
 [     5   5687]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    710294
           1       0.91      1.00      0.95      5692

    accuracy                           1.00    715986
   macro avg       0.95      1.00      0.98    715986
weighted avg       1.00      1.00      1.00    715986



#### 모델 저장

In [ ]:
# !pip install joblib

In [20]:
import joblib

joblib.dump(best_lgbm, 'GSS_LightGBM_model.pkl')
print("Model saved → GSS_LightGBM_model.pkl")
joblib.dump(study, 'optuna_study.pkl')

Model saved → GSS_LightGBM_model.pkl


['optuna_study.pkl']